In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
from pathlib import Path
import openpyxl
import itertools
import torch
import gc
import parquet

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances, make_scorer, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway, friedmanchisquare, wilcoxon
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

Please note that you are missing the optional dependency: fugue. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: snowflake. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: spark. If you need to use this functionality it must be installed.
Python 3.12 and above currently is not supported by Spark and Ray. Please note that some functionality will not work and currently is not supported.


In [2]:
x_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\cicunsw\\Dataset.csv")
y_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\cicunsw\\Label.csv")

In [5]:
df = pd.concat([x_data, y_data], axis=1)

In [7]:
df.describe()

,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,4.479150e+05,447915.000000,447915.000000,4.479150e+05,4.479150e+05,447915.000000,447915.000000,447915.000000,447915.000000,447915.000000,...,447915.000000,4.479150e+05,4.479150e+05,4.479150e+05,4.479150e+05,4.479150e+05,4.479150e+05,4.479150e+05,4.479150e+05,447915.000000
mean,5.983003e+05,22.598428,27.238503,4.778406e+03,2.170251e+04,174.243153,14.918360,71.264324,64.000867,365.009993,...,25.926651,1.868487e+04,9.409518e+03,3.126578e+04,1.252120e+04,9.859284e+04,3.341817e+03,1.026228e+05,9.578854e+04,1.005829
std,4.878777e+06,127.986936,116.638803,1.518561e+05,1.477431e+05,352.171446,71.570572,184.565303,156.387353,555.808533,...,9.017636,7.487275e+05,5.231382e+05,1.174788e+06,6.397847e+05,2.222806e+06,2.201782e+05,2.260988e+06,2.207332e+06,2.095053
min,1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,3.490000e+02,1.000000,2.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,5.788000e+03,3.000000,2.000000,9.000000e+01,1.060000e+02,45.000000,0.000000,9.346154,0.000000,48.000000,...,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
75%,1.792405e+05,17.000000,15.000000,4.860000e+02,1.512000e+03,163.000000,0.000000,50.000000,29.277002,731.000000,...,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
max,1.199975e+08,20038.000000,11021.000000,2.605270e+07,1.421599e+07,1476.000000,1472.000000,1474.000000,848.528137,1460.000000,...,40.000000,1.104063e+08,7.417760e+07,1.104063e+08,1.104063e+08,1.191926e+08,7.214360e+07,1.191926e+08,1.191926e+08,9.000000


In [11]:
df["Label"] = df["Label"].apply(lambda x: 0 if x == 0 else 1)

In [10]:
columns = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',  'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']
scaler = StandardScaler()
df[columns] = scaler.fit_transform(df[columns])

In [13]:
df = df.apply(pd.to_numeric, downcast='integer')
df = df.apply(pd.to_numeric, downcast='float')

In [14]:
df.to_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\original.parquet", compression='snappy', index=False)

In [ ]:
# split data
X = df.drop(columns=['Label'])
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,        # 20% danych do testu
    random_state=42,      # dla powtarzalności
    stratify=y            # zachowaj proporcje klas
)

X_train.shape, X_test.shape

((313540, 76), (134375, 76))

In [17]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

train_df.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_train.parquet', compression='snappy', index=False)
test_df.to_parquet('D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_test.parquet', compression='snappy', index=False)

### oversampling

In [19]:
file_path1 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote_data.csv"
file_path2 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote3_data.csv"
#zamiana jesli istnieje to wczytaj plik jesli nie to stworz
coun = Counter(y_train)
majority_class = max(coun, key=coun.get)
minority_class = min(coun, key=coun.get)
missing_samples = coun[majority_class] - coun[minority_class]
print("Before", coun)
smote = SMOTE()
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

smote2 = SMOTE(sampling_strategy={minority_class: coun[minority_class] + 2 * missing_samples}, random_state=42)
X_train_sm3, y_train_sm3 = smote2.fit_resample(X_train, y_train)

train_data_smote = pd.concat([X_train_sm, y_train_sm], axis=1)          #polaczenie danych wygenerowanych X_train oraz y_train
train_data_smote3 = pd.concat([X_train_sm3, y_train_sm3], axis=1)

#smote generated data
train_data_smote['generated_by_smote'] = ['original' if i < len(train_df) else 'smote' for i in range(len(train_data_smote))]
smote_data = train_data_smote[train_data_smote['generated_by_smote'] == 'smote'].drop('generated_by_smote', axis=1)
smote_data["source"] = "smote"
if not os.path.exists(file_path1):
    smote_data.to_csv(file_path1, index=False)
else:
    print(f'Plik istnieje pod ścieżką: {file_path1}')

#smote3 generated data
train_data_smote3['generated_by_smote'] = ['original' if i < len(train_df) else 'smote' for i in range(len(train_data_smote3))]
smote_data3 = train_data_smote3[train_data_smote3['generated_by_smote'] == 'smote'].drop('generated_by_smote', axis=1)
smote_data3["source"] = "smote"
if not os.path.exists(file_path2):
    smote_data3.to_csv(file_path2, index=False)
else:
    print(f'Plik istnieje pod ścieżką: {file_path2}')

con1 = Counter(y_train_sm)
print("After", con1)
con2 = Counter(y_train_sm3)
print("After generation 3x SMOTE", con2)
#pd.Series(y_train_sm).value_counts().plot.bar()
#pd.Series(y_train_sm3).value_counts().plot.bar()

smote_data.to_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote_generated.parquet", compression='snappy', index=False)
smote_data3.to_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote3_generated.parquet", compression='snappy', index=False)

Before Counter({0: 250832, 1: 62708})


c:\Users\Mati\AppData\Local\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Nie można odnaleźć określonego pliku
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close

After Counter({0: 250832, 1: 250832})
After generation 3x SMOTE Counter({1: 438956, 0: 250832})


In [20]:
file_path2 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline_data.csv"
file_path1 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline3_data.csv"
#generate new data by borderLineSMOTE
coun3 = Counter(y_train)
majority_class = max(coun3, key=coun3.get)
minority_class = min(coun3, key=coun3.get)
missing_samples = coun3[majority_class] - coun3[minority_class]
print("Before", coun3)

brdsmote = BorderlineSMOTE(random_state=42)
X_train_bsm, y_train_bsm = brdsmote.fit_resample(X_train, y_train)

brdsmote3 = BorderlineSMOTE(sampling_strategy={minority_class: coun3[minority_class] + 2 * missing_samples}, random_state=42)
X_train_bsm3, y_train_bsm3 = brdsmote3.fit_resample(X_train, y_train)

con4 = Counter(y_train_bsm)
print("After", con4)
con5 = Counter(y_train_bsm3)
print("After generated 3x BorderlineSMOTE", con5)

train_data_borderline_smote = pd.concat([X_train_bsm, y_train_bsm], axis=1)          #polaczenie danych wygenerowanych X_train oraz y_train
train_data_borderline_smote3 = pd.concat([X_train_bsm3, y_train_bsm3], axis=1)

#borderline smote generated data
train_data_borderline_smote['generated_by_borderline_smote'] = ['original' if i < len(train_df) else 'brd smote' for i in range(len(train_data_borderline_smote))]
boarderline_smote_data = train_data_borderline_smote[train_data_borderline_smote['generated_by_borderline_smote'] == 'brd smote'].drop('generated_by_borderline_smote', axis=1)
boarderline_smote_data["source"]="borderline smote"
boarderline_smote_data = boarderline_smote_data[boarderline_smote_data['source'] != 'original']
if not os.path.exists(file_path2):
    boarderline_smote_data.to_csv(file_path2, index=False)
else:
    print(f'Plik istnieje pod ścieżką: {file_path2}')
    
#borderline smote3 generated data
train_data_borderline_smote3['generated_by_borderline_smote'] = ['original' if i < len(train_df) else 'brd smote' for i in range(len(train_data_borderline_smote3))]
boarderline_smote_data3 = train_data_borderline_smote3[train_data_borderline_smote3['generated_by_borderline_smote'] == 'brd smote'].drop('generated_by_borderline_smote', axis=1)
boarderline_smote_data3["source"]="borderline smote"
boarderline_smote_data3 = boarderline_smote_data3[boarderline_smote_data3['source'] != 'original']
if not os.path.exists(file_path1):
    boarderline_smote_data3.to_csv(file_path1, index=False)
else:
    print(f'Plik istnieje pod ścieżką: {file_path1}')

boarderline_smote_data.to_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline_data.parquet", compression='snappy', index=False)
boarderline_smote_data3.to_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline3_data.parquet", compression='snappy', index=False)


Before Counter({0: 250832, 1: 62708})
After Counter({0: 250832, 1: 250832})
After generated 3x BorderlineSMOTE Counter({1: 438956, 0: 250832})


In [21]:
file_path3 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN_data.csv"        #sciezka wraz z nazwa pod jaka wygenerowac plik
file_path4 = "D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN3_data.csv"        #sciezka wraz z nazwa pod jaka wygenerowac plik

# Chunking parameters
chunk_rows = 30000      # ile wierszy mniejszości na jedną porcję (dostosuj do VRAM/RAM)
max_shards = None       # np. 10 by ograniczyć liczbę porcji; None = bez limitu

# CTGAN
CTGAN_KW = dict(
    epochs=10,
    batch_size=16,
    generator_dim=(64, 64),
    discriminator_dim=(64, 64),
    embedding_dim=16,
    pac=1,
    cuda=True,          # będzie zbijane na False przy OOM
)

# ===== pomocnicze =====
def detect_discrete_columns(df, label_col="Label", max_int_card=50):
    """Zbuduj listę kolumn dyskretnych dla CTGAN (stare API oczekuje takiej listy)."""
    disc = []
    for c in df.columns:
        if c == label_col:
            disc.append(c); continue
        dt = df[c].dtype
        if str(dt) in ("object", "bool") or "category" in str(dt):
            disc.append(c)
        elif np.issubdtype(dt, np.integer) and df[c].nunique(dropna=True) <= max_int_card:
            disc.append(c)
    return sorted(list(set(disc)))

def make_ctgan():
    try:
        from ctgan import CTGAN
    except Exception as e:
        raise RuntimeError("Brak pakietu ctgan. Zainstaluj: pip install ctgan") from e
    return CTGAN(**CTGAN_KW)

def fit_with_fallback(ctgan, data, discrete_columns):
    """Spróbuj na GPU, w razie OOM spadnij na CPU i oczyść pamięć GPU."""
    try:
        ctgan.fit(data, discrete_columns)
        return ctgan
    except RuntimeError as e:
        msg = str(e).lower()
        if "cuda" in msg or "out of memory" in msg or "cublas" in msg:
            # fallback na CPU
            try:
                import torch
                torch.cuda.empty_cache()
            except Exception:
                pass
            # przeinstancjuj CTGAN na CPU
            cpu_kwargs = {**CTGAN_KW, "cuda": False}
            from ctgan import CTGAN
            ctgan_cpu = CTGAN(**cpu_kwargs)
            ctgan_cpu.fit(data, discrete_columns)
            return ctgan_cpu
        raise

def generate_from_shards(minority_df, deficit_total, discrete_columns,
                         multiplier=1, chunk_rows=30000, max_shards=None):
    """
    minority_df: tylko mniejszość (Label == minority_label)
    deficit_total: ile łącznie przykładów musimy dorobić, by zbalansować.
    multiplier: 1 dla 'GAN' (wyrównanie), 2 dla 'GAN3' (×2 mniejszość).
    """
    target_total = int(deficit_total * multiplier)
    if target_total <= 0:
        return pd.DataFrame(columns=minority_df.columns)

    # Pokrój minority na porcje
    n = len(minority_df)
    if chunk_rows <= 0:
        chunk_rows = n  # jedna porcja
    shard_indices = list(range(0, n, chunk_rows))
    if max_shards is not None:
        shard_indices = shard_indices[:max_shards]

    samples_left = target_total
    generated_parts = []

    for i, start in enumerate(shard_indices, 1):
        end = min(start + chunk_rows, n)
        shard = minority_df.iloc[start:end].copy()

        # Rozdziel ile generujemy z tej porcji (proporcjonalnie do rozmiaru porcji)
        # + nigdy nie przekraczaj tego co zostało
        share = (len(shard) / n)
        to_make = int(round(target_total * share))
        to_make = min(to_make, samples_left)
        if to_make <= 0:
            continue

        # Każda porcja: nowy, mały model (czyści pamięć po zakończeniu)
        ctgan = make_ctgan()
        ctgan = fit_with_fallback(ctgan, shard, discrete_columns)

        synth = ctgan.sample(to_make)
        generated_parts.append(synth)

        samples_left -= to_make

        # porządkowanie pamięci
        del ctgan; gc.collect()
        try:
            import torch
            torch.cuda.empty_cache()
        except Exception:
            pass

        if samples_left <= 0:
            break

    if samples_left > 0:
        # Jeżeli zaokrąglenia zostawiły nam ogon — dorób z ostatniej porcji
        last = minority_df.iloc[shard_indices[-1]:min(shard_indices[-1]+chunk_rows, n)].copy()
        ctgan = make_ctgan()
        ctgan = fit_with_fallback(ctgan, last, discrete_columns)
        synth_tail = ctgan.sample(samples_left)
        generated_parts.append(synth_tail)
        del ctgan; gc.collect()
        try:
            import torch
            torch.cuda.empty_cache()
        except Exception:
            pass

    if not generated_parts:
        return pd.DataFrame(columns=minority_df.columns)

    out = pd.concat(generated_parts, ignore_index=True)
    return out

# ===== GŁÓWNY PRZEPŁYW =====

# Zakładam, że masz df w pamięci:
# df = df.drop(columns=["Unnamed: 0"], errors="ignore")
df_gan = train_df

label_col = "Label"
counts = df_gan[label_col].value_counts().sort_index()
if set(counts.index) != {0, 1}:
    raise ValueError(f"Oczekiwałem klas 0/1 w kolumnie '{label_col}', mam: {counts.index.tolist()}")

majority_label = counts.idxmax()
minority_label = 1 - majority_label
deficit = counts[majority_label] - counts[minority_label]

# Kolumny dyskretne do CTGAN
discrete_columns = detect_discrete_columns(df_gan, label_col=label_col, max_int_card=50)

# Dane mniejszości
minority_df = df_gan[df_gan[label_col] == minority_label].reset_index(drop=True)

# --- Wersja GAN (wyrównanie 1:1) ---
df_GAN = generate_from_shards(
    minority_df=minority_df,
    deficit_total=deficit,
    discrete_columns=discrete_columns,
    multiplier=1,
    chunk_rows=chunk_rows,
    max_shards=max_shards
)

balanced_data = pd.concat([df_gan, df_GAN], ignore_index=True)

# --- Wersja GAN3 (×2 mniejszość) ---
df_GAN3 = generate_from_shards(
    minority_df=minority_df,
    deficit_total=deficit,
    discrete_columns=discrete_columns,
    multiplier=2,
    chunk_rows=chunk_rows,
    max_shards=max_shards
)

balanced_data3 = pd.concat([df_gan, df_GAN3], ignore_index=True)

# Raporty
y_train = df_gan[label_col]                    # Twoje "Before" z oryginału
y_train_gan  = balanced_data[label_col]
y_train_gan3 = balanced_data3[label_col]

print("Before:", Counter(y_train))
print("After (GAN):", Counter(y_train_gan))
print("After (GAN3):", Counter(y_train_gan3))

# Oznacz generatywne
df_GAN["source"]  = "gan"
df_GAN3["source"] = "gan"

# Zapis
if not os.path.exists(file_path3):
    df_GAN.to_csv(file_path3, index=False)
else:
    print(f"Plik istnieje pod ścieżką: {file_path3}")

if not os.path.exists(file_path4):
    df_GAN3.to_csv(file_path4, index=False)
else:
    print(f"Plik istnieje pod ścieżką: {file_path4}")

df_GAN.to_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN_generated.parquet", compression='snappy', index=False)
df_GAN3.to_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN3_generated.parquet", compression='snappy', index=False)

Before: Counter({0: 250832, 1: 62708})
After (GAN): Counter({0: 250832, 1: 250832})
After (GAN3): Counter({1: 438956, 0: 250832})
